<a href="https://colab.research.google.com/github/MagnateEmpiror26/video_frame_classifier_tunga_mack/blob/main/shonapredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Shona Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 file = open("/content/drive/MyDrive/kurumwanechekucheraDS.txt","r")
data_set = file.read()

In [ ]:
data_set

'Murooraaa!\nTauya naye!\nMurooraaa!\nTauya nayee\nMurooraa\nTauya naye\nnemagumbezee\nMudunhu rekwaChitororo yaiva\nburushashike vanhu vachifarira muroora\nmutsva anga awuya mumusha\nwekwaNhoto. Ivo vaNhoto vainzwa\nmanyuku nyuku nekuti mwanakomana\nwavo mukuru Wayne anga avaunzira\nndangaruvazhe.\nWayne anga avakushanda mudhorobha\nriripedyo nedunhu ravo. Kubva\npaakapedza zvidzidzo zvake zveOlevel\nakabva angoenda kudhorobha\nkunoshava. Akabva awana basa\nkwaPower Sales. Ikoko kudhorobha ndiko\nkwaakasangana netsvarakadenga yake\nyainzi Rosemary Chuma.\nVakafambidzana mwedzi yakati kuti\nkusvika vaviri ava vapanana pamuviri.\nVakabva vatoronga zvekutizisana.\nMaChuma akabva atora vana tete vake\nvaviri vakava ndivo vakamuperedza.\nPavakasvika mudunhu raChitororo\nvakambogara pamba pabamunini\nvaWayne mazuva matatu muroora asati\naperekwa mumusha memurume wake.\nZuva rekuperekwa kwake rakazosvika.\nMunhu wese mudunhu iri anga\nakarimirira zuva iri. Dunhu rese\nrakaturunuka kuzoona sv

### Defining Vocabulary

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data_set])
total_words = len(tokenizer.word_index) + 1




### Word Embeddings

In [ ]:
from gensim.models import Word2Vec
sentences = [sentence.split() for sentence in data_set.split('.')]
model_gensim = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
model_gensim.save("word2vec.model")


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

model1 = Sequential()
model1.add(Embedding(total_words, 100, input_length=5))
model1.add(Bidirectional(LSTM(150, return_sequences=True)))
model1.add(Dropout(0.3))
model1.add(LSTM(100))
model1.add(Dense(total_words, activation='softmax'))

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


##### :

In [ ]:

import numpy as np
model_gensim = Word2Vec.load("word2vec.model")
embedding_matrix = np.zeros((total_words, 100))
for word, i in tokenizer.word_index.items():
    try:
        embedding_vector = model_gensim.wv[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    except KeyError:

        pass


model2 = Sequential()
model2.add(Embedding(total_words, 100, weights=[embedding_matrix], input_length=5, trainable=False))
model2.add(Bidirectional(LSTM(150, return_sequences=True)))
model2.add(Dropout(0.2))
model2.add(LSTM(100))
model2.add(Dense(total_words, activation='softmax'))

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


###  Train  Test

In [ ]:
import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

input_sequences = []
for line in data_set.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

input_sequences = np.array(pad_sequences(input_sequences, maxlen=6, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = keras.utils.to_categorical(y, num_classes=total_words)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42)

# Training Model 1
history1 = model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1)

# Training Model 2
history2 = model2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1)

Epoch 1/100
30/30 [==============================] - 13s 183ms/step - loss: 7.0701 - accuracy: 0.0063 - val_loss: 7.0724 - val_accuracy: 0.0032
Epoch 2/100
30/30 [==============================] - 4s 117ms/step - loss: 6.8736 - accuracy: 0.0126 - val_loss: 7.6219 - val_accuracy: 0.0237
Epoch 3/100
30/30 [==============================] - 2s 57ms/step - loss: 6.5467 - accuracy: 0.0148 - val_loss: 8.0459 - val_accuracy: 0.0205
Epoch 4/100
30/30 [==============================] - 2s 55ms/step - loss: 6.4116 - accuracy: 0.0148 - val_loss: 8.3642 - val_accuracy: 0.0158
Epoch 5/100
30/30 [==============================] - 2s 59ms/step - loss: 6.2748 - accuracy: 0.0169 - val_loss: 8.6668 - val_accuracy: 0.0110
Epoch 6/100
30/30 [==============================] - 2s 56ms/step - loss: 6.0714 - accuracy: 0.0158 - val_loss: 8.9909 - val_accuracy: 0.0142
Epoch 7/100
30/30 [==============================] - 2s 57ms/step - loss: 5.8830 - accuracy: 0.0200 - val_loss: 9.6209 - val_accuracy: 0.0142
Epo

### Model Evaluation

In [ ]:
val_loss_model1 = history1.history['val_loss'][-1]
val_loss_model2 = history2.history['val_loss'][-1]

print(f"Validation Loss for Model 1: {val_loss_model1}")
print(f"Validation Loss for Model 2: {val_loss_model2}")


Validation Loss for Model 1: 13.6737699508667
Validation Loss for Model 2: 14.056363105773926


In [ ]:
if val_loss_model1 < val_loss_model2:
    best_model = model1
    best_model_name = "best_model1.h5"
else:
    best_model = model2
    best_model_name = "best_model2.h5"

best_model.save(best_model_name)
print(f"Saved the best model as {best_model_name}")


Saved the best model as best_model1.h5


In [ ]:
params_model1 = model1.count_params()
params_model2 = model2.count_params()

print(f"Model 1 has {params_model1} parameters.")
print(f"Model 2 has {params_model2} parameters.")


Model 1 has 698177 parameters.
Model 2 has 698177 parameters.


In [ ]:
from keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the previously saved model
model = load_model('best_model1.h5')

def predict_next_words(model, tokenizer, text, num_words=1):
    """
    Predict the next set of words using the trained model.

    Args:
    - model (keras.Model): The trained model.
    - tokenizer (Tokenizer): The tokenizer object used for preprocessing.
    - text (str): The input text.
    - num_words (int): The number of words to predict.

    Returns:
    - str: The predicted words.
    """
    for _ in range(num_words):

        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = pad_sequences([sequence], maxlen=5, padding='pre')


        predicted_probs = model.predict(sequence, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)


        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break


        text += " " + output_word

    return ' '.join(text.split(' ')[-num_words:])


# Ask the user for input
user_input = input("Input 5 Shona Words: ")


predicted_words = predict_next_words(model, tokenizer, user_input, num_words=3)
print(f"The next words might be: {predicted_words}")


Input 5 Shona Words: tinodzidza zuva nezuva zvinoita tipase
The next words might be: mugouya vatanga kunyomba
